# Group Surveys Feature Determination

## Preparation

### Import

In [28]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 1000)

## Function Definition

### Read and Concatenate Tables

In [30]:
def read_collapse_tables(whats_features, meetings_features, surveys_features, grades):

  df_whats= pd.read_csv(whats_features, index_col='id',delimiter=";", header=0)
  df_meetings= pd.read_excel(meetings_features,header=0,skiprows=range(1, 4)).rename_axis('Id')
  df_meetings.index += 1
  df_meetings= df_meetings.drop(columns=['group','shown_face','ID'], axis=1)
  df_surveys= pd.read_csv(surveys_features, index_col=0)
  df_surveys= df_surveys.drop(columns=['group'], axis=1)
  df_grades= pd.read_excel(grades, index_col='ID', usecols=['ID','TOTAL TEORÍA (6)',	'COEVAL (1)',	'TRABAJO FINAL (1)'])
  df_grades.columns=['theory','coeval','project']
  df = pd.concat([df_meetings, df_whats, df_surveys, df_grades], axis=1)

  return df

In [31]:
def clean_dataset(df):

  # Print the 'name' column for rows that are dropped due to filtering
  dropped_names_dropna = df[df['theory'].isna()]['name']
  dropped_names_filter = df[df['theory'] < 0.5]['name']

  print("Dropped Rows (dropna):")
  print(dropped_names_dropna)

  print("Dropped Rows (filtering):")
  print(dropped_names_filter)

  # Apply the filtering conditions
  df = df.dropna()
  df = df[df['theory'] >= 0.5]

  return df


### Display Scatter Plot

In [32]:
def scatterplot (df,x_column,y_column):
  import pandas as pd
  import seaborn as sns
  import matplotlib.pyplot as plt
  import numpy as np
  from sklearn.linear_model import LinearRegression
  from sklearn.preprocessing import PolynomialFeatures

  # Assuming your DataFrame is called 'df' and you want to check relationships between columns 'x' and 'y'
  x = df[x_column]
  y = df[y_column]

  # Create a scatter plot to visualize the relationship
  plt.scatter(x, y)
  plt.xlabel('x')
  plt.ylabel('y')
  plt.title('Scatter Plot')
  plt.show()

  # Fit a polynomial regression model
  degree = 2  # Degree of the polynomial
  poly_features = PolynomialFeatures(degree=degree)
  X_poly = poly_features.fit_transform(x.values.reshape(-1, 1))

  model = LinearRegression()
  model.fit(X_poly, y)

  # Calculate predicted values
  y_pred = model.predict(X_poly)

  # Plot the fitted curve
  plt.scatter(x, y)
  plt.plot(x, y_pred, color='red', label=f'Degree {degree} Polynomial Fit')
  plt.xlabel('x')
  plt.ylabel('y')
  plt.title('Polynomial Regression')
  plt.legend()
  plt.show()


### Correlation Significance

In [33]:
def check_correlation_significance(df, col1, col2):

  # Extract the 'performance' and 'recreational_perceived_mean' columns
  performance = df[col1]
  recreational_mean = df[col2]

  # Perform the hypothesis test
  corr_coeff, p_value = stats.pearsonr(performance, recreational_mean)

  # Print the results
  print("Correlation coefficient:", corr_coeff)
  print("p-value:", p_value)

  # Compare the p-value to the significance level
  alpha = 0.05  # Significance level
  if p_value < alpha:
      print("The correlation is statistically significant (reject H0).")
  else:
      print("The correlation is not statistically significant (fail to reject H0).")


## Use of Function

In [34]:
grades = r'/content/drive/MyDrive/Projects/tps/grades/data/1_participants_grades.xlsx'
whats_features= r'/content/drive/MyDrive/Projects/tps/whatsapp/data/1_nodes_mixed.csv'
meetings_features= r'/content/drive/MyDrive/Projects/tps/meetings/data/12. features/4_individual_features_final_ratio.xlsx'
surveys_features=  r'/content/drive/MyDrive/Projects/tps/surveys/data/1_happimeter_individual_surveys.csv'

In [35]:
df1= pd.read_csv('nodes.csv')
df1.head(70)

,id,messages_sent,messages_received,out_degree,in_degree,degree,messages_total,contribution_index,ego_art,ego_nudges,alter_art,alter_nudges,sentiment_avg,emotionality_avg,complexity_avg,influence_total,influence_message_avg_in,influence_message_avg,influence_total_in,cc_ratio_sent,cc_ratio_received,cc_avg
0,carmen,8,29,6,6,12,37,-0.57,0.073556,1.946667,0.071181,2.145833,0.371270,0.223186,9.729522,0.514820,0.188724,0.405159,0.188724,0.0,0.0,6.0
1,javier uni,44,257,5,5,10,301,-0.71,7.393083,3.609414,7.504956,2.525147,0.338921,0.200355,8.846823,0.806041,0.253938,0.465257,0.790506,0.0,0.0,5.0
2,miriam,8,18,5,5,10,26,-0.38,0.633333,2.000000,0.258333,4.625000,0.452997,0.180231,8.389843,0.055376,0.077513,0.055376,0.077513,0.0,0.0,5.0
3,isma uni,6,29,6,6,12,35,-0.66,0.900000,3.000000,0.197917,1.875000,0.574343,0.170770,10.155345,2.123331,0.000000,0.530833,0.000000,0.0,0.0,6.0
4,raúl galera,27,82,4,4,8,109,-0.50,2.206481,2.561111,1.083602,3.578022,0.319963,0.264397,9.177844,0.858611,0.228996,0.412297,0.599066,0.0,0.0,4.0
5,tomasz,73,346,6,6,12,419,-0.65,1.004476,3.105521,3.266492,3.311011,0.440138,0.224488,5.899958,2.323498,0.749370,0.847271,3.191054,0.0,0.0,6.0
6,+34 654 55 66 09,4,18,5,5,10,22,-0.64,0.527778,4.000000,0.283333,2.000000,0.466168,0.217933,7.655167,1.116796,0.088933,1.116796,0.088933,0.0,0.0,5.0
7,cesar calidad g elias,38,153,5,5,10,191,-0.60,10.063711,2.010089,9.903273,2.270121,0.381860,0.254014,9.034121,1.038947,0.199154,0.236068,0.866594,0.0,0.0,5.0
8,alejandro,20,68,5,5,10,88,-0.55,17.443750,1.540909,6.056944,1.479167,0.306511,0.187171,7.979794,0.945774,0.103944,0.254012,0.519722,0.0,0.0,5.0
9,luna calidad software,27,153,5,5,10,180,-0.70,11.976921,2.333088,7.537312,1.568277,0.250852,0.259828,9.557120,1.159895,0.467754,0.486947,2.016371,0.0,0.0,5.0


In [36]:
df= read_collapse_tables(whats_features, meetings_features, surveys_features, grades)
df.head(70)

,indiv_spoken_time,indiv_spoken_time_ratio,average_turn_duration,average_turn_duration_ratio,avg_time_without_speaking,avg_time_without_speaking_ratio,max_time_without_speaking,max_time_without_speaking_ratio,num_turns,num_turns_ratio,avg_turns_without_speaking,avg_turns_without_speaking_ratio,max_turns_without_speaking,max_turns_without_speaking_ratio,num_words,num_words_ratio,avg_words_turn,avg_words_turn_ratio,max_words_turn,max_words_turn_ratio,speech_neu,speech_ang,speech_hap,speech_sad,text_joy,text_anger,text_fear,text_sadness,group,name,messages_sent,messages_received,out_degree,in_degree,degree,messages_total,contribution_index,ego_art,ego_nudges,alter_art,alter_nudges,sentiment_avg,emotionality_avg,complexity_avg,influence_message_avg,influence_total_in,influence_message_avg_in,influence_total,contribution_index_oscillation,activity_entanglement,ALTERNATIVE_REALITIES_Treehugger,ALTERNATIVE_REALITIES_Fatherlander,ALTERNATIVE_REALITIES_Spiritualism,ALTERNATIVE_REALITIES_Nerd,ALTERNATIVE_REALITIES,IDEOLOGY_Liberalism,IDEOLOGY_Complainers,IDEOLOGY_Capitalism,IDEOLOGY_Socialism,IDEOLOGY,RECREATION_Sport,RECREATION_Fashion,RECREATION_Travel,RECREATION_Arts,RECREATION,LIFESTYLES_Vegan,LIFESTYLES_Sedentary,LIFESTYLES_Yolo,LIFESTYLES_Fitness,LIFESTYLES,PERSONALITY_Journalist,PERSONALITY_Risk-taker,PERSONALITY_Stock-trader,PERSONALITY_Politician,PERSONALITY,EMOTIONS_Fear,EMOTIONS_Happy,EMOTIONS_Sad,EMOTIONS_Anger,EMOTIONS,Behavioral_Values_Interest_Disinterest,Behavioral_Values_Interest_Interest,Behavioral_Values_Interest,Behavioral_Values_Modesty_Modesty,Behavioral_Values_Modesty_Arrogance,Behavioral_Values_Modesty,Behavioral_Values_Fairness_Unfairness,Behavioral_Values_Fairness_Fairness,Behavioral_Values_Fairness,Groupflow_Beeflow,Groupflow_Leechflow,Groupflow_Antflow,Groupflow,EMOTIONS_ALL_Fear,EMOTIONS_ALL_Amazement,EMOTIONS_ALL_Anger,EMOTIONS_ALL_Happy,EMOTIONS_ALL_Admiration,EMOTIONS_ALL_Disgust,EMOTIONS_ALL_Neutral,EMOTIONS_ALL_Anticipation,EMOTIONS_ALL_Sad,EMOTIONS_ALL,Mail,Id_happimeter,id_dospert,timestamp_dospert,ethical_likelihood,financial_likelihood,health_likelihood,recreational_likelihood,social_likelihood,total_likelihood,ethical_perceived,financial_perceived,health_perceived,recreational_perceived,social_perceived,total_perceived,id_ffi,timestamp_ffi,O,C,E,A,N,id_morals,timestamp_morals,harm_care_score,fairness_reciprocity_score,in_group_loyality_score,authority_respect_score,purity_sanctity_score,dummy_question1,dummy_question2,id_schwartz,timestamp_schwartz,q1,q2,q3,q4,q5,q6,q7,q8,q9,q10,conservation,transcendence,theory,coeval,project
1,526.0,0.148336,7.850746,0.126865,56.210746,0.165958,451.54,0.264343,67.0,0.331683,1.985075,0.093801,7.0,0.090909,978.0,0.170800,14.597015,0.127952,68.0,0.078613,0.832695,0.137502,0.026347,0.003456,0.298809,0.236814,0.232361,0.232015,1.0,robert,34.0,153.0,5.0,5.0,10.0,187.0,-0.64,1.332.855.889,2.468.578.279,1.234.255.833,1.903.921.545,0.303561,0.215562,9.179.501.577,0.346125353,1.227.594.788,0.346135338,1.730.626.767,1.0,0.487649,4.720845e-01,8.388939e-02,0.089140,3.548859e-01,Treehugger,1.307350e-01,0.687836,1.476230e-02,1.666667e-01,Complainers,1.002846e-01,0.270154,0.437937,0.191625,Travel,0.061560,0.032021,0.874533,0.031885,Yolo,1.230000e-04,0.868930,1.308545e-01,9.190000e-05,Risk-taker,0.236101,0.407447,0.251288,0.105165,Happy,0.631772,0.368228,Disinterest,0.605879,0.394121,Modesty,0.446794,0.553206,Fairness,0.497477,0.156358,0.346165,Beeflow,1.566476e-01,1.090000e-05,2.940000e-09,1.298257e-01,2.860000e-09,1.720000e-08,0.663721,6.440000e-09,4.979453e-02,Neutral,robert.banu@alumnos.upm.es,2688,259.0,12/15/2022 15:21,4.500000,3.500000,4.166667,3.166667,3.500000,3.766667,4.833333,5.666667,5.666667,5.833333,4.833333,5.0,578.0,12/15/2022 15:13,0.600000,0.716667,0.600000,0.633333,0.583333,292.0,12/15/2022 15:30,27.0,23.0,19.0,18.0,20.0,1.0,5.0,354.0,12/15/2022 15:23,7.0,6.0,5.0,7.0,5.0,4.0,5.0,7.0,8.0,7.0,2.57,-1.78,3.44,0.90,8.0
2,2648.0,0.746757,33.948718,0.548598,21.791169

In [37]:
df.shape

(63, 152)

In [38]:
rows_with_nan = df[df.isna().any(axis=1)]

rows_with_nan.head(8)

,indiv_spoken_time,indiv_spoken_time_ratio,average_turn_duration,average_turn_duration_ratio,avg_time_without_speaking,avg_time_without_speaking_ratio,max_time_without_speaking,max_time_without_speaking_ratio,num_turns,num_turns_ratio,avg_turns_without_speaking,avg_turns_without_speaking_ratio,max_turns_without_speaking,max_turns_without_speaking_ratio,num_words,num_words_ratio,avg_words_turn,avg_words_turn_ratio,max_words_turn,max_words_turn_ratio,speech_neu,speech_ang,speech_hap,speech_sad,text_joy,text_anger,text_fear,text_sadness,group,name,messages_sent,messages_received,out_degree,in_degree,degree,messages_total,contribution_index,ego_art,ego_nudges,alter_art,alter_nudges,sentiment_avg,emotionality_avg,complexity_avg,influence_message_avg,influence_total_in,influence_message_avg_in,influence_total,contribution_index_oscillation,activity_entanglement,ALTERNATIVE_REALITIES_Treehugger,ALTERNATIVE_REALITIES_Fatherlander,ALTERNATIVE_REALITIES_Spiritualism,ALTERNATIVE_REALITIES_Nerd,ALTERNATIVE_REALITIES,IDEOLOGY_Liberalism,IDEOLOGY_Complainers,IDEOLOGY_Capitalism,IDEOLOGY_Socialism,IDEOLOGY,RECREATION_Sport,RECREATION_Fashion,RECREATION_Travel,RECREATION_Arts,RECREATION,LIFESTYLES_Vegan,LIFESTYLES_Sedentary,LIFESTYLES_Yolo,LIFESTYLES_Fitness,LIFESTYLES,PERSONALITY_Journalist,PERSONALITY_Risk-taker,PERSONALITY_Stock-trader,PERSONALITY_Politician,PERSONALITY,EMOTIONS_Fear,EMOTIONS_Happy,EMOTIONS_Sad,EMOTIONS_Anger,EMOTIONS,Behavioral_Values_Interest_Disinterest,Behavioral_Values_Interest_Interest,Behavioral_Values_Interest,Behavioral_Values_Modesty_Modesty,Behavioral_Values_Modesty_Arrogance,Behavioral_Values_Modesty,Behavioral_Values_Fairness_Unfairness,Behavioral_Values_Fairness_Fairness,Behavioral_Values_Fairness,Groupflow_Beeflow,Groupflow_Leechflow,Groupflow_Antflow,Groupflow,EMOTIONS_ALL_Fear,EMOTIONS_ALL_Amazement,EMOTIONS_ALL_Anger,EMOTIONS_ALL_Happy,EMOTIONS_ALL_Admiration,EMOTIONS_ALL_Disgust,EMOTIONS_ALL_Neutral,EMOTIONS_ALL_Anticipation,EMOTIONS_ALL_Sad,EMOTIONS_ALL,Mail,Id_happimeter,id_dospert,timestamp_dospert,ethical_likelihood,financial_likelihood,health_likelihood,recreational_likelihood,social_likelihood,total_likelihood,ethical_perceived,financial_perceived,health_perceived,recreational_perceived,social_perceived,total_perceived,id_ffi,timestamp_ffi,O,C,E,A,N,id_morals,timestamp_morals,harm_care_score,fairness_reciprocity_score,in_group_loyality_score,authority_respect_score,purity_sanctity_score,dummy_question1,dummy_question2,id_schwartz,timestamp_schwartz,q1,q2,q3,q4,q5,q6,q7,q8,q9,q10,conservation,transcendence,theory,coeval,project
7,1369.0,0.343452,10.779528,0.235076,24.375748,0.086970,482.10,0.126133,127.0,0.292627,2.401575,0.088632,46.0,0.147436,4522.0,0.369747,35.606299,0.261726,253.0,0.256853,0.765414,0.215585,0.016789,0.002213,0.293694,0.239227,0.229892,0.237187,2.0,alejandro bueno,5.0,47.0,5.0,5.0,10.0,52.0,-0.81,0.088888889,4.333.333.333,1.1,1.166.666.667,0.530476,0.266764,6.500.736.904,NaN,NaN,NaN,NaN,3.0,0.572801,0.199388,0.200054,0.000418,0.600141,Nerd,0.000003,0.999858,0.000002,0.000137,Complainers,0.267894,0.215834,0.200826,0.315446,Arts,0.003074,0.077523,0.905643,0.013760,Yolo,8.769732e-02,0.400357,5.119402e-01,5.160000e-06,Stock-trader,0.386561,0.281953,0.204929,0.126557,Fear,0.998901,0.001099,Disinterest,0.201691,0.798309,Arrogance,0.612315,0.387685,Unfairness,0.009022,0.021835,0.969143,Antflow,0.591481,3.140000e-05,5.390000e-09,0.022966,1.150000e-09,4.500000e-09,0.210212,5.450000e-10,0.175310,Fear,alejandro.bueno.perez@alumnos.upm.es,2727,269.0,12/17/2022 16:09,2.833333,3.166667,1.666667,2.166667,5.666667,3.100000,3.0,4.500000,5.833333,5.166667,2.833333,4.0,588.0,12/17/2022 16:00,0.583333,0.683333,0.650000,0.616667,0.550000,301.0,12/17/2022 16:48,21.0,22.0,19.0,14.0,10.0,0.0,5.0,363.0,12/17/2022 16:19,4.0,6.0,4.0,5.0,7.0,3.0,5.0,3.0,6.0,6.0,1.34,-1.40,3.08,0.77,8.5
28,23.0,0.048117,2.090909,0.066399,45.156364,0.300430,155.86,0.321839,11.0,0.150685,5.090909,0.283143,19.0,0.322034,93.0,0.

In [39]:
dropped_names_filter = df[df['theory'] < 0.5]
dropped_names_filter.head()

,indiv_spoken_time,indiv_spoken_time_ratio,average_turn_duration,average_turn_duration_ratio,avg_time_without_speaking,avg_time_without_speaking_ratio,max_time_without_speaking,max_time_without_speaking_ratio,num_turns,num_turns_ratio,avg_turns_without_speaking,avg_turns_without_speaking_ratio,max_turns_without_speaking,max_turns_without_speaking_ratio,num_words,num_words_ratio,avg_words_turn,avg_words_turn_ratio,max_words_turn,max_words_turn_ratio,speech_neu,speech_ang,speech_hap,speech_sad,text_joy,text_anger,text_fear,text_sadness,group,name,messages_sent,messages_received,out_degree,in_degree,degree,messages_total,contribution_index,ego_art,ego_nudges,alter_art,alter_nudges,sentiment_avg,emotionality_avg,complexity_avg,influence_message_avg,influence_total_in,influence_message_avg_in,influence_total,contribution_index_oscillation,activity_entanglement,ALTERNATIVE_REALITIES_Treehugger,ALTERNATIVE_REALITIES_Fatherlander,ALTERNATIVE_REALITIES_Spiritualism,ALTERNATIVE_REALITIES_Nerd,ALTERNATIVE_REALITIES,IDEOLOGY_Liberalism,IDEOLOGY_Complainers,IDEOLOGY_Capitalism,IDEOLOGY_Socialism,IDEOLOGY,RECREATION_Sport,RECREATION_Fashion,RECREATION_Travel,RECREATION_Arts,RECREATION,LIFESTYLES_Vegan,LIFESTYLES_Sedentary,LIFESTYLES_Yolo,LIFESTYLES_Fitness,LIFESTYLES,PERSONALITY_Journalist,PERSONALITY_Risk-taker,PERSONALITY_Stock-trader,PERSONALITY_Politician,PERSONALITY,EMOTIONS_Fear,EMOTIONS_Happy,EMOTIONS_Sad,EMOTIONS_Anger,EMOTIONS,Behavioral_Values_Interest_Disinterest,Behavioral_Values_Interest_Interest,Behavioral_Values_Interest,Behavioral_Values_Modesty_Modesty,Behavioral_Values_Modesty_Arrogance,Behavioral_Values_Modesty,Behavioral_Values_Fairness_Unfairness,Behavioral_Values_Fairness_Fairness,Behavioral_Values_Fairness,Groupflow_Beeflow,Groupflow_Leechflow,Groupflow_Antflow,Groupflow,EMOTIONS_ALL_Fear,EMOTIONS_ALL_Amazement,EMOTIONS_ALL_Anger,EMOTIONS_ALL_Happy,EMOTIONS_ALL_Admiration,EMOTIONS_ALL_Disgust,EMOTIONS_ALL_Neutral,EMOTIONS_ALL_Anticipation,EMOTIONS_ALL_Sad,EMOTIONS_ALL,Mail,Id_happimeter,id_dospert,timestamp_dospert,ethical_likelihood,financial_likelihood,health_likelihood,recreational_likelihood,social_likelihood,total_likelihood,ethical_perceived,financial_perceived,health_perceived,recreational_perceived,social_perceived,total_perceived,id_ffi,timestamp_ffi,O,C,E,A,N,id_morals,timestamp_morals,harm_care_score,fairness_reciprocity_score,in_group_loyality_score,authority_respect_score,purity_sanctity_score,dummy_question1,dummy_question2,id_schwartz,timestamp_schwartz,q1,q2,q3,q4,q5,q6,q7,q8,q9,q10,conservation,transcendence,theory,coeval,project
29,70.0,0.146444,4.666667,0.148194,22.052857,0.146720,61.52,0.127034,15.0,0.205479,2.200000,0.122358,7.0,0.118644,256.0,0.161209,17.066667,0.163169,37.0,0.132143,0.816990,0.172084,0.010692,0.000234,0.298994,0.238890,0.224710,0.237406,6.0,carlos ads,17.0,82.0,5.0,5.0,10.0,99.0,-0.66,0.572797622,3.928.571.463,0.889270833,2.715.972.185,0.217433,0.237678,8.497.999.042,0.260869739,1.015.778.301,0.330908499,0.810207018,1.0,0.413315,0.557965,0.08810,0.060458,0.293477,Treehugger,0.121635,0.844078,0.000106,0.034181,Complainers,0.084502,0.162071,0.330301,0.423126,Arts,0.001770,0.057137,0.937102,0.003991,Yolo,4.930000e-07,0.941062,0.058938,1.630000e-07,Risk-taker,0.316839,0.293867,0.297044,0.092250,Fear,0.875344,0.124656,Disinterest,0.351900,0.648100,Arrogance,0.623405,0.376595,Unfairness,0.398358,0.000526,0.601116,Antflow,0.123080,0.000023,2.620000e-08,0.017364,1.750000e-08,1.060000e-07,0.705798,3.170000e-07,0.153734,Neutral,carlos.teva.moreno@alumnos.upm.es,2904,327.0,1/31/2023 1:34,4.0,4.000000,4.0,4.0,4.000000,4.000000,4.666667,3.666667,3.333333,4.333333,3.833333,4.0,662.0,1/31/2023 1:32,0.633333,0.616667,0.65,0.566667,0.550000,357.0,1/31/2023 1:35,13.0,18.0,17.0,12.0,15.0,2.0,2.0,420.0,1/31/2023 1:34,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,1.26,-1.16,0.0,0.87,8.7
59,1150.0,0.230553,8.846154,0.176243,37.228000,0.125245,310.46,0.123775,130.0,0.219224,3.553846,0.125580,27.0,0.128571,2

In [40]:
import pandas as pd

def add_columns_to_df_by_code(df1, df2, id1_column, id2_column, columns):
    # Merge df1 and df2 based on the matching IDs
    merged_df = pd.merge(df1, df2[[id2_column] + columns], left_on=id1_column, right_on=id2_column, how='left')

    # Rename the newly added columns to include '_df2' as a suffix
    for column in columns:
        merged_df.rename(columns={column: f'{column}_df2'}, inplace=True)

    # Drop the duplicate ID column from df2
    merged_df.drop(id2_column, axis=1, inplace=True)

    return merged_df



# Specify the columns you want to copy from df2 to df1
columns_to_copy = ['ego_art', 'ego_nudges', 'alter_art','alter_nudges','influence_message_avg','influence_total_in','influence_message_avg_in','influence_total']

# Add the specified columns from df2 to df1 where IDs match
result_df = add_columns_to_df_by_code(df, df1, 'name', 'id', columns_to_copy)

In [41]:
result_df.head(70)

,indiv_spoken_time,indiv_spoken_time_ratio,average_turn_duration,average_turn_duration_ratio,avg_time_without_speaking,avg_time_without_speaking_ratio,max_time_without_speaking,max_time_without_speaking_ratio,num_turns,num_turns_ratio,avg_turns_without_speaking,avg_turns_without_speaking_ratio,max_turns_without_speaking,max_turns_without_speaking_ratio,num_words,num_words_ratio,avg_words_turn,avg_words_turn_ratio,max_words_turn,max_words_turn_ratio,speech_neu,speech_ang,speech_hap,speech_sad,text_joy,text_anger,text_fear,text_sadness,group,name,messages_sent,messages_received,out_degree,in_degree,degree,messages_total,contribution_index,ego_art_x,ego_nudges_x,alter_art_x,alter_nudges_x,sentiment_avg,emotionality_avg,complexity_avg,influence_message_avg_x,influence_total_in_x,influence_message_avg_in_x,influence_total_x,contribution_index_oscillation,activity_entanglement,ALTERNATIVE_REALITIES_Treehugger,ALTERNATIVE_REALITIES_Fatherlander,ALTERNATIVE_REALITIES_Spiritualism,ALTERNATIVE_REALITIES_Nerd,ALTERNATIVE_REALITIES,IDEOLOGY_Liberalism,IDEOLOGY_Complainers,IDEOLOGY_Capitalism,IDEOLOGY_Socialism,IDEOLOGY,RECREATION_Sport,RECREATION_Fashion,RECREATION_Travel,RECREATION_Arts,RECREATION,LIFESTYLES_Vegan,LIFESTYLES_Sedentary,LIFESTYLES_Yolo,LIFESTYLES_Fitness,LIFESTYLES,PERSONALITY_Journalist,PERSONALITY_Risk-taker,PERSONALITY_Stock-trader,PERSONALITY_Politician,PERSONALITY,EMOTIONS_Fear,EMOTIONS_Happy,EMOTIONS_Sad,EMOTIONS_Anger,EMOTIONS,Behavioral_Values_Interest_Disinterest,Behavioral_Values_Interest_Interest,Behavioral_Values_Interest,Behavioral_Values_Modesty_Modesty,Behavioral_Values_Modesty_Arrogance,Behavioral_Values_Modesty,Behavioral_Values_Fairness_Unfairness,Behavioral_Values_Fairness_Fairness,Behavioral_Values_Fairness,Groupflow_Beeflow,Groupflow_Leechflow,Groupflow_Antflow,Groupflow,EMOTIONS_ALL_Fear,EMOTIONS_ALL_Amazement,EMOTIONS_ALL_Anger,EMOTIONS_ALL_Happy,EMOTIONS_ALL_Admiration,EMOTIONS_ALL_Disgust,EMOTIONS_ALL_Neutral,EMOTIONS_ALL_Anticipation,EMOTIONS_ALL_Sad,EMOTIONS_ALL,Mail,Id_happimeter,id_dospert,timestamp_dospert,ethical_likelihood,financial_likelihood,health_likelihood,recreational_likelihood,social_likelihood,total_likelihood,ethical_perceived,financial_perceived,health_perceived,recreational_perceived,social_perceived,total_perceived,id_ffi,timestamp_ffi,O,C,E,A,N,id_morals,timestamp_morals,harm_care_score,fairness_reciprocity_score,in_group_loyality_score,authority_respect_score,purity_sanctity_score,dummy_question1,dummy_question2,id_schwartz,timestamp_schwartz,q1,q2,q3,q4,q5,q6,q7,q8,q9,q10,conservation,transcendence,theory,coeval,project,ego_art_y,ego_nudges_y,alter_art_y,alter_nudges_y,influence_message_avg_y,influence_total_in_y,influence_message_avg_in_y,influence_total_y
0,526.0,0.148336,7.850746,0.126865,56.210746,0.165958,451.54,0.264343,67.0,0.331683,1.985075,0.093801,7.0,0.090909,978.0,0.170800,14.597015,0.127952,68.0,0.078613,0.832695,0.137502,0.026347,0.003456,0.298809,0.236814,0.232361,0.232015,1.0,robert,34.0,153.0,5.0,5.0,10.0,187.0,-0.64,1.332.855.889,2.468.578.279,1.234.255.833,1.903.921.545,0.303561,0.215562,9.179.501.577,0.346125353,1.227.594.788,0.346135338,1.730.626.767,1.0,0.487649,4.720845e-01,8.388939e-02,0.089140,3.548859e-01,Treehugger,1.307350e-01,0.687836,1.476230e-02,1.666667e-01,Complainers,1.002846e-01,0.270154,0.437937,0.191625,Travel,0.061560,0.032021,0.874533,0.031885,Yolo,1.230000e-04,0.868930,1.308545e-01,9.190000e-05,Risk-taker,0.236101,0.407447,0.251288,0.105165,Happy,0.631772,0.368228,Disinterest,0.605879,0.394121,Modesty,0.446794,0.553206,Fairness,0.497477,0.156358,0.346165,Beeflow,1.566476e-01,1.090000e-05,2.940000e-09,1.298257e-01,2.860000e-09,1.720000e-08,0.663721,6.440000e-09,4.979453e-02,Neutral,robert.banu@alumnos.upm.es,2688,259.0,12/15/2022 15:21,4.500000,3.500000,4.166667,3.166667,3.500000,3.766667,4.833333,5.666667,5.666667,5.833333,4.833333,5.0,578.0,12/15/2022 15:13,0.600000,0.716667,0.600000,0.633333,0.583333,292.0,12/15/2022 15:30,27.0,23.0,1

In [42]:
result_df.shape

(63, 160)

In [43]:
rows_with_nan = result_df[result_df.isna().any(axis=1)]

rows_with_nan.head(8)

,indiv_spoken_time,indiv_spoken_time_ratio,average_turn_duration,average_turn_duration_ratio,avg_time_without_speaking,avg_time_without_speaking_ratio,max_time_without_speaking,max_time_without_speaking_ratio,num_turns,num_turns_ratio,avg_turns_without_speaking,avg_turns_without_speaking_ratio,max_turns_without_speaking,max_turns_without_speaking_ratio,num_words,num_words_ratio,avg_words_turn,avg_words_turn_ratio,max_words_turn,max_words_turn_ratio,speech_neu,speech_ang,speech_hap,speech_sad,text_joy,text_anger,text_fear,text_sadness,group,name,messages_sent,messages_received,out_degree,in_degree,degree,messages_total,contribution_index,ego_art_x,ego_nudges_x,alter_art_x,alter_nudges_x,sentiment_avg,emotionality_avg,complexity_avg,influence_message_avg_x,influence_total_in_x,influence_message_avg_in_x,influence_total_x,contribution_index_oscillation,activity_entanglement,ALTERNATIVE_REALITIES_Treehugger,ALTERNATIVE_REALITIES_Fatherlander,ALTERNATIVE_REALITIES_Spiritualism,ALTERNATIVE_REALITIES_Nerd,ALTERNATIVE_REALITIES,IDEOLOGY_Liberalism,IDEOLOGY_Complainers,IDEOLOGY_Capitalism,IDEOLOGY_Socialism,IDEOLOGY,RECREATION_Sport,RECREATION_Fashion,RECREATION_Travel,RECREATION_Arts,RECREATION,LIFESTYLES_Vegan,LIFESTYLES_Sedentary,LIFESTYLES_Yolo,LIFESTYLES_Fitness,LIFESTYLES,PERSONALITY_Journalist,PERSONALITY_Risk-taker,PERSONALITY_Stock-trader,PERSONALITY_Politician,PERSONALITY,EMOTIONS_Fear,EMOTIONS_Happy,EMOTIONS_Sad,EMOTIONS_Anger,EMOTIONS,Behavioral_Values_Interest_Disinterest,Behavioral_Values_Interest_Interest,Behavioral_Values_Interest,Behavioral_Values_Modesty_Modesty,Behavioral_Values_Modesty_Arrogance,Behavioral_Values_Modesty,Behavioral_Values_Fairness_Unfairness,Behavioral_Values_Fairness_Fairness,Behavioral_Values_Fairness,Groupflow_Beeflow,Groupflow_Leechflow,Groupflow_Antflow,Groupflow,EMOTIONS_ALL_Fear,EMOTIONS_ALL_Amazement,EMOTIONS_ALL_Anger,EMOTIONS_ALL_Happy,EMOTIONS_ALL_Admiration,EMOTIONS_ALL_Disgust,EMOTIONS_ALL_Neutral,EMOTIONS_ALL_Anticipation,EMOTIONS_ALL_Sad,EMOTIONS_ALL,Mail,Id_happimeter,id_dospert,timestamp_dospert,ethical_likelihood,financial_likelihood,health_likelihood,recreational_likelihood,social_likelihood,total_likelihood,ethical_perceived,financial_perceived,health_perceived,recreational_perceived,social_perceived,total_perceived,id_ffi,timestamp_ffi,O,C,E,A,N,id_morals,timestamp_morals,harm_care_score,fairness_reciprocity_score,in_group_loyality_score,authority_respect_score,purity_sanctity_score,dummy_question1,dummy_question2,id_schwartz,timestamp_schwartz,q1,q2,q3,q4,q5,q6,q7,q8,q9,q10,conservation,transcendence,theory,coeval,project,ego_art_y,ego_nudges_y,alter_art_y,alter_nudges_y,influence_message_avg_y,influence_total_in_y,influence_message_avg_in_y,influence_total_y
6,1369.0,0.343452,10.779528,0.235076,24.375748,0.086970,482.10,0.126133,127.0,0.292627,2.401575,0.088632,46.0,0.147436,4522.0,0.369747,35.606299,0.261726,253.0,0.256853,0.765414,0.215585,0.016789,0.002213,0.293694,0.239227,0.229892,0.237187,2.0,alejandro bueno,5.0,47.0,5.0,5.0,10.0,52.0,-0.81,0.088888889,4.333.333.333,1.1,1.166.666.667,0.530476,0.266764,6.500.736.904,NaN,NaN,NaN,NaN,3.0,0.572801,0.199388,0.200054,0.000418,0.600141,Nerd,0.000003,0.999858,0.000002,0.000137,Complainers,0.267894,0.215834,0.200826,0.315446,Arts,0.003074,0.077523,0.905643,0.013760,Yolo,8.769732e-02,0.400357,5.119402e-01,5.160000e-06,Stock-trader,0.386561,0.281953,0.204929,0.126557,Fear,0.998901,0.001099,Disinterest,0.201691,0.798309,Arrogance,0.612315,0.387685,Unfairness,0.009022,0.021835,0.969143,Antflow,0.591481,3.140000e-05,5.390000e-09,0.022966,1.150000e-09,4.500000e-09,0.210212,5.450000e-10,0.175310,Fear,alejandro.bueno.perez@alumnos.upm.es,2727,269.0,12/17/2022 16:09,2.833333,3.166667,1.666667,2.166667,5.666667,3.100000,3.0,4.500000,5.833333,5.166667,2.833333,4.0,588.0,12/17/2022 16:00,0.583333,0.683333,0.650000,0.616667,0.550000,301.0,12/17/2022 16:48,21.0,22.0,19.0,14.0,10.0,0.0,5.0,363.0,12/17/2022 16:19,4.0,6.0,4.0,5.0,7.0,3.0,5.0,3.0,

In [44]:
columns_to_show=['ego_art_y', 'ego_nudges_y', 'alter_art_y','alter_nudges_y','influence_message_avg_y','influence_total_in_y','influence_message_avg_in_y','influence_total_y']

In [45]:
result_df[columns_to_show].head(70)

,ego_art_y,ego_nudges_y,alter_art_y,alter_nudges_y,influence_message_avg_y,influence_total_in_y,influence_message_avg_in_y,influence_total_y
0,13.328559,2.468578,12.342558,1.903922,0.346125,1.227595,0.346135,1.730627
1,7.778627,1.363152,9.695611,3.791191,0.428516,0.997036,0.231981,1.988108
2,10.063711,2.010089,9.903273,2.270121,0.236068,0.866594,0.199154,1.038947
3,11.976921,2.333088,7.537312,1.568277,0.486947,2.016371,0.467754,1.159895
4,8.437014,2.777083,12.106078,1.418481,0.360360,1.780057,0.421674,0.970076
5,13.264359,1.628846,5.609306,4.304167,0.275772,0.030909,0.030909,0.412770
6,0.088889,4.333333,1.100000,1.166667,NaN,NaN,NaN,NaN
7,3.302708,3.037500,14.760833,2.912500,0.000000,0.959899,0.959899,0.000000
8,20.955833,3.866667,16.238194,2.291667,0.061817,0.254339,0.127169,0.061817
9,12.132917,1.720833,9.400256,2.871795,0.037611,0.098446,0.098446,0.037611


In [46]:
#df = min_max_scaling_df(df)
df.rename_axis('Id', inplace=True)
df.head(70)

,indiv_spoken_time,indiv_spoken_time_ratio,average_turn_duration,average_turn_duration_ratio,avg_time_without_speaking,avg_time_without_speaking_ratio,max_time_without_speaking,max_time_without_speaking_ratio,num_turns,num_turns_ratio,avg_turns_without_speaking,avg_turns_without_speaking_ratio,max_turns_without_speaking,max_turns_without_speaking_ratio,num_words,num_words_ratio,avg_words_turn,avg_words_turn_ratio,max_words_turn,max_words_turn_ratio,speech_neu,speech_ang,speech_hap,speech_sad,text_joy,text_anger,text_fear,text_sadness,group,name,messages_sent,messages_received,out_degree,in_degree,degree,messages_total,contribution_index,ego_art,ego_nudges,alter_art,alter_nudges,sentiment_avg,emotionality_avg,complexity_avg,influence_message_avg,influence_total_in,influence_message_avg_in,influence_total,contribution_index_oscillation,activity_entanglement,ALTERNATIVE_REALITIES_Treehugger,ALTERNATIVE_REALITIES_Fatherlander,ALTERNATIVE_REALITIES_Spiritualism,ALTERNATIVE_REALITIES_Nerd,ALTERNATIVE_REALITIES,IDEOLOGY_Liberalism,IDEOLOGY_Complainers,IDEOLOGY_Capitalism,IDEOLOGY_Socialism,IDEOLOGY,RECREATION_Sport,RECREATION_Fashion,RECREATION_Travel,RECREATION_Arts,RECREATION,LIFESTYLES_Vegan,LIFESTYLES_Sedentary,LIFESTYLES_Yolo,LIFESTYLES_Fitness,LIFESTYLES,PERSONALITY_Journalist,PERSONALITY_Risk-taker,PERSONALITY_Stock-trader,PERSONALITY_Politician,PERSONALITY,EMOTIONS_Fear,EMOTIONS_Happy,EMOTIONS_Sad,EMOTIONS_Anger,EMOTIONS,Behavioral_Values_Interest_Disinterest,Behavioral_Values_Interest_Interest,Behavioral_Values_Interest,Behavioral_Values_Modesty_Modesty,Behavioral_Values_Modesty_Arrogance,Behavioral_Values_Modesty,Behavioral_Values_Fairness_Unfairness,Behavioral_Values_Fairness_Fairness,Behavioral_Values_Fairness,Groupflow_Beeflow,Groupflow_Leechflow,Groupflow_Antflow,Groupflow,EMOTIONS_ALL_Fear,EMOTIONS_ALL_Amazement,EMOTIONS_ALL_Anger,EMOTIONS_ALL_Happy,EMOTIONS_ALL_Admiration,EMOTIONS_ALL_Disgust,EMOTIONS_ALL_Neutral,EMOTIONS_ALL_Anticipation,EMOTIONS_ALL_Sad,EMOTIONS_ALL,Mail,Id_happimeter,id_dospert,timestamp_dospert,ethical_likelihood,financial_likelihood,health_likelihood,recreational_likelihood,social_likelihood,total_likelihood,ethical_perceived,financial_perceived,health_perceived,recreational_perceived,social_perceived,total_perceived,id_ffi,timestamp_ffi,O,C,E,A,N,id_morals,timestamp_morals,harm_care_score,fairness_reciprocity_score,in_group_loyality_score,authority_respect_score,purity_sanctity_score,dummy_question1,dummy_question2,id_schwartz,timestamp_schwartz,q1,q2,q3,q4,q5,q6,q7,q8,q9,q10,conservation,transcendence,theory,coeval,project
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,526.0,0.148336,7.850746,0.126865,56.210746,0.165958,451.54,0.264343,67.0,0.331683,1.985075,0.093801,7.0,0.090909,978.0,0.170800,14.597015,0.127952,68.0,0.078613,0.832695,0.137502,0.026347,0.003456,0.298809,0.236814,0.232361,0.232015,1.0,robert,34.0,153.0,5.0,5.0,10.0,187.0,-0.64,1.332.855.889,2.468.578.279,1.234.255.833,1.903.921.545,0.303561,0.215562,9.179.501.577,0.346125353,1.227.594.788,0.346135338,1.730.626.767,1.0,0.487649,4.720845e-01,8.388939e-02,0.089140,3.548859e-01,Treehugger,1.307350e-01,0.687836,1.476230e-02,1.666667e-01,Complainers,1.002846e-01,0.270154,0.437937,0.191625,Travel,0.061560,0.032021,0.874533,0.031885,Yolo,1.230000e-04,0.868930,1.308545e-01,9.190000e-05,Risk-taker,0.236101,0.407447,0.251288,0.105165,Happy,0.631772,0.368228,Disinterest,0.605879,0.394121,Modesty,0.446794,0.553206,Fairness,0.497477,0.156358,0.346165,Beeflow,1.566476e-01,1.090000e-05,2.940000e-09,1.298257e-01,2.860000e-09,1.720000e-08,0.663721,6.440000e-09,4.979453e-02,Neutral,robert.banu@alumnos.upm.es,2688,259.0,12/15/2022 15:21,4.500000,3.500000,4.166667,3.166667,3.500000,3.766667,4.833333,5.666667,5.666667,5.833333,4.833333,5.0,578.0,12/15/2022 15:13,0.600000,0.716667,0.600000,0.633333,0.583333,292.0,12/15/2022 15:30,27.0,23.0,19

In [47]:
columns_to_remove= ['name','Mail','Id_happimeter','id_dospert','timestamp_dospert','id_ffi','timestamp_ffi','id_morals','timestamp_morals','id_schwartz','timestamp_schwartz']
df= df.drop(columns= columns_to_remove)

In [48]:
df.shape

(63, 141)

In [49]:
for c in df.columns:
  print(c)

indiv_spoken_time
indiv_spoken_time_ratio
average_turn_duration
average_turn_duration_ratio
avg_time_without_speaking
avg_time_without_speaking_ratio
max_time_without_speaking
max_time_without_speaking_ratio
num_turns
num_turns_ratio
avg_turns_without_speaking
avg_turns_without_speaking_ratio
max_turns_without_speaking
max_turns_without_speaking_ratio
num_words
num_words_ratio
avg_words_turn
avg_words_turn_ratio
max_words_turn
max_words_turn_ratio
speech_neu
speech_ang
speech_hap
speech_sad
text_joy
text_anger
text_fear
text_sadness
group
messages_sent
messages_received
out_degree
in_degree
degree
messages_total
contribution_index
ego_art
ego_nudges
alter_art
alter_nudges
sentiment_avg
emotionality_avg
complexity_avg
influence_message_avg
influence_total_in
influence_message_avg_in
influence_total
contribution_index_oscillation
activity_entanglement
ALTERNATIVE_REALITIES_Treehugger
ALTERNATIVE_REALITIES_Fatherlander
ALTERNATIVE_REALITIES_Spiritualism
ALTERNATIVE_REALITIES_Nerd
ALTERNAT

In [50]:
df.head()

,indiv_spoken_time,indiv_spoken_time_ratio,average_turn_duration,average_turn_duration_ratio,avg_time_without_speaking,avg_time_without_speaking_ratio,max_time_without_speaking,max_time_without_speaking_ratio,num_turns,num_turns_ratio,avg_turns_without_speaking,avg_turns_without_speaking_ratio,max_turns_without_speaking,max_turns_without_speaking_ratio,num_words,num_words_ratio,avg_words_turn,avg_words_turn_ratio,max_words_turn,max_words_turn_ratio,speech_neu,speech_ang,speech_hap,speech_sad,text_joy,text_anger,text_fear,text_sadness,group,messages_sent,messages_received,out_degree,in_degree,degree,messages_total,contribution_index,ego_art,ego_nudges,alter_art,alter_nudges,sentiment_avg,emotionality_avg,complexity_avg,influence_message_avg,influence_total_in,influence_message_avg_in,influence_total,contribution_index_oscillation,activity_entanglement,ALTERNATIVE_REALITIES_Treehugger,ALTERNATIVE_REALITIES_Fatherlander,ALTERNATIVE_REALITIES_Spiritualism,ALTERNATIVE_REALITIES_Nerd,ALTERNATIVE_REALITIES,IDEOLOGY_Liberalism,IDEOLOGY_Complainers,IDEOLOGY_Capitalism,IDEOLOGY_Socialism,IDEOLOGY,RECREATION_Sport,RECREATION_Fashion,RECREATION_Travel,RECREATION_Arts,RECREATION,LIFESTYLES_Vegan,LIFESTYLES_Sedentary,LIFESTYLES_Yolo,LIFESTYLES_Fitness,LIFESTYLES,PERSONALITY_Journalist,PERSONALITY_Risk-taker,PERSONALITY_Stock-trader,PERSONALITY_Politician,PERSONALITY,EMOTIONS_Fear,EMOTIONS_Happy,EMOTIONS_Sad,EMOTIONS_Anger,EMOTIONS,Behavioral_Values_Interest_Disinterest,Behavioral_Values_Interest_Interest,Behavioral_Values_Interest,Behavioral_Values_Modesty_Modesty,Behavioral_Values_Modesty_Arrogance,Behavioral_Values_Modesty,Behavioral_Values_Fairness_Unfairness,Behavioral_Values_Fairness_Fairness,Behavioral_Values_Fairness,Groupflow_Beeflow,Groupflow_Leechflow,Groupflow_Antflow,Groupflow,EMOTIONS_ALL_Fear,EMOTIONS_ALL_Amazement,EMOTIONS_ALL_Anger,EMOTIONS_ALL_Happy,EMOTIONS_ALL_Admiration,EMOTIONS_ALL_Disgust,EMOTIONS_ALL_Neutral,EMOTIONS_ALL_Anticipation,EMOTIONS_ALL_Sad,EMOTIONS_ALL,ethical_likelihood,financial_likelihood,health_likelihood,recreational_likelihood,social_likelihood,total_likelihood,ethical_perceived,financial_perceived,health_perceived,recreational_perceived,social_perceived,total_perceived,O,C,E,A,N,harm_care_score,fairness_reciprocity_score,in_group_loyality_score,authority_respect_score,purity_sanctity_score,dummy_question1,dummy_question2,q1,q2,q3,q4,q5,q6,q7,q8,q9,q10,conservation,transcendence,theory,coeval,project
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,526.0,0.148336,7.850746,0.126865,56.210746,0.165958,451.54,0.264343,67.0,0.331683,1.985075,0.093801,7.0,0.090909,978.0,0.170800,14.597015,0.127952,68.0,0.078613,0.832695,0.137502,0.026347,0.003456,0.298809,0.236814,0.232361,0.232015,1.0,34.0,153.0,5.0,5.0,10.0,187.0,-0.64,1.332.855.889,2.468.578.279,1.234.255.833,1.903.921.545,0.303561,0.215562,9.179.501.577,0.346125353,1.227.594.788,0.346135338,1.730.626.767,1.0,0.487649,0.472084,0.083889,0.089140,0.354886,Treehugger,0.130735,0.687836,0.014762,0.166667,Complainers,0.100285,0.270154,0.437937,0.191625,Travel,0.061560,0.032021,0.874533,0.031885,Yolo,0.000123,0.868930,0.130854,0.000092,Risk-taker,0.236101,0.407447,0.251288,0.105165,Happy,0.631772,0.368228,Disinterest,0.605879,0.394121,Modesty,0.446794,0.553206,Fairness,0.497477,0.156358,0.346165,Beeflow,0.156648,0.000011,2.940000e-09,0.129826,2.860000e-09,1.720000e-08,0.663721,6.440000e-09,0.049795,Neutral,4.500000,3.500000,4.166667,3.166667,3.500000,3.766667,4.833333,5.666667,5.666667,5.833333,4.833333,5.0,0.600000,0.716667,0.600000,0.633333,0.583333,27.0,23.0,19.0,18.0,20.0,1.0,5.0,7.0,6.0,5.0,7.0,5.0,4.0,5.0,7.0,8.0,7.0,2.57,-1.78,3.44,0.90,8.0
2,2648.0,0.746757,33.948718,0.548598,21.791169,0.064337,109.24,0.063952,78.0,0.386139,1.589744,0.075120,9.0,0.116883,3816.0,0.666434,48.923077,0.428842,623.0,0.720231,0.795958,0.155654,0.045315,0.003074,0.292950,0.239695,

In [51]:
df.dtypes

indiv_spoken_time                         float64
indiv_spoken_time_ratio                   float64
average_turn_duration                     float64
average_turn_duration_ratio               float64
avg_time_without_speaking                 float64
avg_time_without_speaking_ratio           float64
max_time_without_speaking                 float64
max_time_without_speaking_ratio           float64
num_turns                                 float64
num_turns_ratio                           float64
avg_turns_without_speaking                float64
avg_turns_without_speaking_ratio          float64
max_turns_without_speaking                float64
max_turns_without_speaking_ratio          float64
num_words                                 float64
num_words_ratio                           float64
avg_words_turn                            float64
avg_words_turn_ratio                      float64
max_words_turn                            float64
max_words_turn_ratio                      float64


In [52]:
out_file= r'/content/drive/MyDrive/Projects/tps/finals/data/3_individual_features_extended.xlsx'
#df.to_excel(out_file)